AI21BTECH11028 
SHREYAS WANKHEDE 
ASSIGNMENT 2

In [1205]:

from torchvision import transforms,datasets
import numpy as np
from PIL import Image
from math import floor
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA

In [1206]:
# defining the sigmoid activation function
def sigmoid(x):
    return (1/(1+np.exp(-1*x)))

In [1207]:
#defining the softmax function
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


PART 1: CONVOLUTION OPERATION

In [1208]:
#convolution function that takes image,kernel,stride.padding , no. of filters as arguments
def convolution_op(input_img,filter,s,p,d,sigmoid):
    # creating output array of necessary dimensions
    out=np.empty((d,int(((input_img.shape[1]-filter.shape[1]+2*p)/s)+1),int(((input_img.shape[2]-filter.shape[2]+2*p)/s)+1)))
    
    #padding 
    new_inp_img=np.zeros((filter.shape[0],input_img.shape[1]+2*p,input_img.shape[2]+2*p))
    for k in range(filter.shape[0]):
     for i in range(p,input_img.shape[1]-p):
       for j in range(p,input_img.shape[2]-p):
           new_inp_img[k][i][j]=input_img[k][i-p][j-p]
    
    #convolution operation
    
    for k in range(d):  #iterating for no. of filters      
      for i in range(0,new_inp_img.shape[1],s):  #moving along height
        for j in range(0,new_inp_img.shape[2],s):  #moving along width
           sum=0
           if(i+filter.shape[1]>new_inp_img.shape[1] or j+filter.shape[2]>new_inp_img.shape[2]):
              sum=0
              break
           else:
              #summing up the results 
              new_sum=0
              for l in range(filter.shape[0]):
               sum=np.sum(np.multiply(new_inp_img[l,i:(filter.shape[1]+i),j:(filter.shape[2]+j)],filter[l]))
               new_sum+=sum
           #storing result in output array
           if(i+filter.shape[1]<=new_inp_img.shape[1] and j+filter.shape[2]<=new_inp_img.shape[2]):
              out[(k,int(i/s),int(j/s))]=new_sum
    #generating activation map         
    act_map=sigmoid(out)
    return act_map


PART 2 : POOLING FUNCTION

In [1209]:
#pooling functions that take activation map,kernel,stride,no. of filters as arguments
def avg_pool(map,kernel_2,s,d):
    #creating output array of required dimensions
    output=np.empty((d,int(map.shape[1]/kernel_2.shape[1]),int(map.shape[2]/kernel_2.shape[2])))
    
    
    for i in range(0,map.shape[1],s):#moving along height
        for j in range(0,map.shape[2],s):#moving along width
            if(i+kernel_2.shape[1]>map.shape[1] or j+kernel_2.shape[2]>map.shape[2]):
                break
            else: 
                #pooling the map
                for k in range(d):
                  sum_inp=np.sum(np.multiply(map[k,i:kernel_2.shape[1]+i,j:kernel_2.shape[2]+j],kernel_2[k]))
            #storing result in the output array
            if(i+kernel_2.shape[1]<=map.shape[1] and j+kernel_2.shape[2]<=map.shape[2]):
                output[(k,int(i/s),int(j/s))]=float(sum_inp/(kernel_2.shape[1]*kernel_2.shape[2]))
    return output
                

def max_pool(map,kernel_2,s,d):
    output=np.empty((d,int(map.shape[1]/kernel_2.shape[1]),int(map.shape[2]/kernel_2.shape[2])))
    
    for i in range(0,map.shape[1],s):
        for j in range(0,map.shape[2],s):
            if(i+kernel_2.shape[1]>map.shape[1] or j+kernel_2.shape[2]>map.shape[2]):
                break
            else: 
                for k in range(d):
                 sum_inp=np.max(np.multiply(map[k,i:(kernel_2.shape[1]+i),j:(kernel_2.shape[2]+j)],kernel_2[k]))
            if(i+kernel_2.shape[1]<=map.shape[1] and j+kernel_2.shape[2]<=map.shape[2]):
                output[(k,int(i/s),int(j/s))]=float(sum_inp)
    return output

def global_avg_pooling(input_vol):
  out_vol=np.empty((1,input_vol.shape[0]))
  for i in range(input_vol.shape[0]):
    out_vol[(1,i)]=np.average(input_vol[i])
  return out_vol
    
    

PART 3: CONVOLUTION LAYER FUNCTION

In [1210]:
#convolution layer function that takes image,no. of filters,kernel dimensions,stride, padding as arguments
def convolution_layer(img,no_filter,ker_dim_1,ker_dim_2,ker_dim_3,strd,pad):
    #creating a kernel of necessary dimensions
    kernel=np.random.rand(ker_dim_1,ker_dim_2,ker_dim_3)
    #calling the convolution function
    return convolution_op(img,kernel,strd,pad,no_filter,sigmoid)

PART 4 : POOLING LAYER FUNCTION

In [1211]:
#pooling layer function with activation map,stride,type of pooling function as arguments
def pooling_layer(input_map,stride,avg_pool_fun:bool=True):
  #creating kernel of necsarry dimensions
    kernel=np.random.rand(input_map.shape[0],stride,stride)
    #calling the pooling function
    if avg_pool_fun==True:
      return avg_pool(input_map,kernel,stride,3)
    else:
      return max_pool(input_map,kernel,stride,3)

PART 5 : FLATTEN FUNCTION

In [1212]:
#flatten function that takes input volume ,specified output size as arguments
def flatten(inp_volume,specified_size):
    array=inp_volume
    #total no. of elements in the input volume
    res_dim=array.shape[0]*array.shape[1]*array.shape[2]
    #reshaping the array
    flat_array=array.reshape(1,res_dim)
    #random weigth matrix for matrix multiplication giving out of size 1x(specified size)
    weight_mat=np.random.rand(flat_array.shape[1],specified_size)
    #matrix multiplication
    result_matrix=np.matmul(flat_array,weight_mat)
    return result_matrix


PART 6 : MLP FUNCTION

In [1213]:
#MLP function that takes input vector. no. of hidden layers ,size of each hidden layer,no. of output nodes, boolean softmax functiion as arguments
def MLP(inp_vector,no_hidden,size_hidden,out_size,sigmoid,softmax_fun:bool):
    array=inp_vector
    #creating random weight matrix for first step of forward pass between input vector and first hidden layer
    wt_matrix=np.random.rand(inp_vector.shape[1],size_hidden)
    #matrix multipliaction
    first_out=np.matmul(array,wt_matrix)
    first_res=sigmoid(first_out)
    
    #genearting weight matrices and carrying out forward pass between hidden layers 
    for i in range(no_hidden-1):
        new_wt_matrix=np.random.rand(size_hidden,size_hidden)
        res_mat=np.matmul(first_res,new_wt_matrix)
        first_res=sigmoid(res_mat)
     
    #forward pass for last step   
    final_wt_mat=np.random.rand(size_hidden,out_size)
    final_out=np.matmul(first_res,final_wt_mat)  
    final_res=sigmoid(final_out)
    
    #putting output in softmax function according to boolean value
    if softmax_fun==True:
       soft=softmax(final_res)
       return soft
    else:
      return(final_res)


PART 7 : FEED FORWARD PATH

In [1214]:
#feed forward path that takes input image and output dimensions as arguments
def feed_forward(input,output_dim,bottleneck:bool=True):
  res_1=convolution_layer(input,16,input.shape[0],3,3,1,1)#calling convolution layer function
  res_2=pooling_layer(res_1,2,False)#calling pooling layer function
  res_3=convolution_layer(res_2,8,res_2.shape[0],3,3,2,1)#again convolution layer function
  res_4=pooling_layer(res_3,2,False)#agian pooling layer function
  res_5=flatten(res_4,10)# flatten function
  if (bottleneck):
    bottleneck_layer=res_5
    return MLP(res_5,1,10,output_dim,sigmoid,softmax_fun=True),bottleneck_layer#bottlneck,MLP(res_5,1,10,output_dim,softmax_fun=True)#
  return MLP(res_5,1,10,output_dim,sigmoid,softmax_fun=True)#returning the MLP output



DOWNLOADING THE DATASET

In [1215]:
dataset=datasets.CIFAR10(root='./data',train=False,download=True,transform=transforms.ToTensor())
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)


Files already downloaded and verified


 IMPLEMENTING FEED FORWARD PATH FOR AN IMAGE

In [1216]:
dataset=np.array(dataset)
output=feed_forward(input=dataset[1][0],output_dim=10,bottleneck=False)
print("output vector = ",output)

/tmp/ipykernel_5539/2296116046.py:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  dataset=np.array(dataset)
/tmp/ipykernel_5539/2296116046.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dataset=np.array(dataset)


output vector =  [[0.09936655 0.10064495 0.09909822 0.10036792 0.10053505 0.10055357
  0.09974195 0.09930156 0.09984056 0.10054966]]


PART 8a : FEEDFORWARD PATH FOR SINGLE IMAGE BELONGING TO EVERY LABEL

In [1217]:

for i in range(10):
    input_dat=dataset[dataset[:,1]==i][0]
    output=feed_forward(input=input_dat[0],output_dim=10)
    print(f"output vector for label {i+1} -\n",output)


output vector for label 1 -
 (array([[0.10009885, 0.10050199, 0.1004218 , 0.10026083, 0.09876198,
        0.10050321, 0.09900182, 0.10024808, 0.10016705, 0.10003439]]), array([[4.18994821, 4.39779774, 5.00306539, 4.57060876, 3.13259948,
        4.60561489, 4.12731853, 4.7350865 , 3.74108539, 4.92964591]]))
output vector for label 2 -
 (array([[0.10034507, 0.10044161, 0.09946499, 0.10033211, 0.10039185,
        0.09981876, 0.09987481, 0.10034803, 0.09890052, 0.10008225]]), array([[6.1498718 , 6.57345156, 5.10655265, 5.51037423, 5.4354885 ,
        6.66411321, 5.99130322, 5.85661389, 4.14195689, 5.39476567]]))
output vector for label 3 -
 (array([[0.10043719, 0.09793961, 0.10042322, 0.09703861, 0.10095696,
        0.10082321, 0.10072295, 0.10074175, 0.1005342 , 0.10038231]]), array([[7.59838795, 6.80332206, 5.95289404, 6.76737876, 7.04857605,
        4.66144056, 5.52730595, 6.4781673 , 7.57709685, 6.43728652]]))
output vector for label 4 -
 (array([[0.10022665, 0.10035883, 0.10032653, 0.

PART 8b : BOTTLENECK LAYER 

In [1218]:
new_data=[]

for i in range(10):
    new_inp=dataset[dataset[:,1]==i][:3]
    
    for j in range(3):
        new_out,bottleneck_layer=feed_forward(input=new_inp[j][0],output_dim=10,bottleneck=True)
        bottleneck_layer=np.array(bottleneck_layer)
        new_data.append(bottleneck_layer)
new_data=np.array(new_data)
PCA_return=PCA(n_components=2)
new_data=PCA_return.fit_transform(new_data)

ValueError: Found array with dim 3. PCA expected <= 2.

In [ ]:
plt.plot(new_data[:,0],new_data[:,1],'ro')
plt.grid()
plt.show()